In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
# raw_documents = TextLoader("tagged_description.txt").load()
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
document = text_splitter.split_documents(raw_documents)

Created a chunk of size 981, which is longer than the specified 0
Created a chunk of size 1055, which is longer than the specified 0
Created a chunk of size 855, which is longer than the specified 0
Created a chunk of size 983, which is longer than the specified 0
Created a chunk of size 1287, which is longer than the specified 0
Created a chunk of size 1103, which is longer than the specified 0
Created a chunk of size 1370, which is longer than the specified 0
Created a chunk of size 1264, which is longer than the specified 0
Created a chunk of size 1231, which is longer than the specified 0
Created a chunk of size 865, which is longer than the specified 0
Created a chunk of size 1118, which is longer than the specified 0
Created a chunk of size 975, which is longer than the specified 0
Created a chunk of size 1245, which is longer than the specified 0
Created a chunk of size 1249, which is longer than the specified 0
Created a chunk of size 1053, which is longer than the specified 0


In [4]:
document[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"s1 Dick Johnson Is Dead is a 2020 movie from United States directed by Kirsten Johnson with a runtime of 90 min (Rated PG-13) Genres: Documentaries Dick Johnson Is Dead is a 2020 American documentary film directed by Kirsten Johnson.  It\'s a poignant and darkly humorous exploration of mortality, focusing on Johnson\'s father, Dick, who is suffering from dementia.  The film playfully imagines various ways Dick might die, staging these scenarios with his enthusiastic participation.  These fictional scenes are interwoven with genuine footage of their tender father-daughter relationship, creating a unique blend of fact and fiction.  The documentary tackles themes of aging, loss, and grief with a surprising grace and subversive humor, ultimately celebrating life and the unique bond between Dick and Kirsten.  Critically acclaimed, the film received a 100% rating on Rotten Tomatoes and an 89 Metascore, earning praise for 

In [5]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.Client()

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = [model.encode(document[index].page_content) for index in range(len(document))]

collection = client.create_collection("netflix-movie-collections-v1")

for i, doc in enumerate(document):

    doc_id = doc.page_content.strip().split()[0]  # Split and take the first word as the document ID
    
    collection.add(
        documents=[doc.page_content],  
        metadatas=[{"source": f"doc{i+1}"}],
        embeddings=[embeddings[i]],
        ids=doc_id  # Pass the extracted document ID
    )

\\vdi-fileshare02\UEMprofiles\033782565\Desktop\NetFlix\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
\\vdi-fileshare02\UEMprofiles\033782565\Desktop\NetFlix\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\033782565\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develop

In [11]:
query = "A movie in which student runs the sex advice clinic in scholl basement"
query_embedding = model.encode([query])

results = collection.query(
    query_embeddings=query_embedding, 
    n_results=10  # Get the top 10 result
)

# Print out the most similar document
# print("Most similar document:", results['documents'])

for result in results['documents'][0]:
    print("movie:" , result)

movie: "s33 Sex Education is a 2020 tv show from United Kingdom starring Asa Butterfield, Gillian Anderson, Ncuti Gatwa, Emma Mackey, Connor Swindells, Kedar Williams-Stirling, Alistair Petrie with a runtime of 3 Seasons (Rated TV-MA) Genres: British TV Shows, International TV Shows, TV Comedies Sex Education is a British teen comedy-drama series created by Laurie Nunn for Netflix.  The series follows the lives of teenagers and adults in the fictional town of Moordale as they navigate various personal dilemmas, often related to sexual intimacy.  The central character is Otis Milburn, a socially awkward, sexually inexperienced teenager whose sex therapist mother inadvertently influences his knowledge of the subject.  Otis and his friend Maeve Wiley create a sex therapy clinic at school to help their fellow students.  The series explores a wide range of sexual issues, including masturbation, relationships, sexual orientation, and abortion, while also delving into broader themes of identi